Name: Parth Singh <br>
PRN: 21070126062 <br>
Batch: AIML A3 <br>
Git: https://github.com/ParthSingh2908/NLP-Assignments.git

In [ ]:
!pip install -U transformers -q
!pip install -U accelerate -q
!pip install keras_nlp -q
!pip install datasets -q
!pip install huggingface-hub -q
!pip install rouge-score -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.1/590.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... 

In [ ]:
# Importing the Needed libraries
import nltk
nltk.download("all",quiet=True)
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
# Loading the dataset
from datasets import load_dataset
dataset = load_dataset("xsum", split="train")
print(dataset)

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 204045
})


In [ ]:
# Printing the dataset
print(dataset[0])

{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

In [ ]:
# Doing the train-test-split
datasets = dataset.train_test_split(train_size=0.05,test_size=0.02)

In [ ]:
# Checking the length of the dataset
print(len(datasets['train']))
print(len(datasets['test']))

10202
4081


In [ ]:
# Define maximum input length for tokenization
MAX_INPUT_LENGTH = 1024

# Define minimum and maximum target lengths for summarization
MIN_TARGET_LENGTH = 5
MAX_TARGET_LENGTH = 128

# Define batch size for training
BATCH_SIZE = 8

# Define learning rate for optimizer
LEARNING_RATE = 2e-5

# Define maximum number of epochs for training
MAX_EPOCHS = 1

# Define the pre-trained T5 model checkpoint to be used
MODEL_CHECKPOINT = "t5-small" # Name of Model


In [ ]:
# Importing the tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

In [ ]:
# Check if the specified model checkpoint is either "t5-small" or "t5-base"
if MODEL_CHECKPOINT in ["t5-small", "t5-base"]:
    # If yes, set the prefix for summarization task
    prefix = "summarize: "
else:
    # If not, set an empty prefix
    prefix = ""


In [ ]:
#Preprocessing
def preprocess_function(examples):
  inputs = [prefix + doc for doc in examples["document"]]
  model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH,truncation=True)
  # Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["summary"], max_length=MAX_TARGET_LENGTH, truncation=True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
# Mapping the preprocessing function to the train and test datasets
train = datasets['train']
test = datasets['test']

In [ ]:
# Mapping the preprocessing function to the train and test datasets
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/10202 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/4081 [00:00<?, ? examples/s]

In [ ]:
# Using tensorflow AutoModelForSeq2SeqLM for summarization
# Using the DataCollatorForSeq2Seq for collating the data
import transformers
from transformers import TFAutoModelForSeq2SeqLM, AutoModelForSeq2SeqLM,DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
import numpy as np
from datasets import load_metric

metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Replace -100 in the predictions as we can't decode them.
    preds = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}


<ipython-input-15-2ba50ed64764>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [ ]:
# Importing Torch
import torch

In [ ]:
# Check if CUDA (GPU support) is available in PyTorch
if torch.cuda.is_available():
    # If CUDA is available, set the device to GPU
    device = torch.device("cuda")
    print("GPU is available and being used")
else:
    # If CUDA is not available, set the device to CPU
    device = torch.device("cpu")
    print("GPU is not available, using CPU instead")



GPU is available and being used


In [ ]:
# Extracting the model name from the MODEL_CHECKPOINT path
model_name = MODEL_CHECKPOINT.split("/")[-1]

# Creating Seq2SeqTrainingArguments with specified settings
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned",  # Output directory for the fine-tuned model
    evaluation_strategy="epoch",  # Evaluation after each epoch
    learning_rate=LEARNING_RATE,  # Learning rate for training
    per_device_train_batch_size=BATCH_SIZE,  # Batch size for training
    per_device_eval_batch_size=BATCH_SIZE,  # Batch size for evaluation
    weight_decay=0.01,  # Weight decay for regularization
    save_total_limit=3,  # Maximum number of checkpoints to keep
    num_train_epochs=MAX_EPOCHS,  # Maximum number of training epochs
    predict_with_generate=True,  # Enable generation during prediction
    fp16=True  # Enable mixed-precision training with 16-bit precision
)


In [ ]:
# Installing or upgrading the 'accelerate' package using pip
!pip install -U accelerate -q

# Importing the 'accelerate' module
import accelerate

# Getting the version of the 'accelerate' module
accelerate.__version__


'0.23.0'

In [ ]:
trainer = Seq2SeqTrainer(
    model.to(device),
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.935300,2.656517,25.130900,5.803100,19.512800,19.501200,18.777500
2,2.836800,2.611203,26.542900,6.514900,20.647900,20.641000,18.818200
3,2.782400,2.585763,26.856900,6.889900,21.005700,21.007100,18.795600
4,2.760700,2.568604,27.099600,7.018700,21.186900,21.182200,18.801800
5,2.713000,2.555967,27.378500,7.211900,21.420300,21.414500,18.813000
6,2.691000,2.546688,27.680600,7.349000,21.653500,21.642500,18.821900
7,2.647900,2.539366,27.862300,7.515700,21.819700,21.813500,18.800800
8,2.628200,2.530066,28.078300,7.600500,21.950600,21.931700,18.814500
9,2.634000,2.526372,28.200600,7.691800,22.143700,22.134100,18.807200
10,2.610600,2.519985,28.317900,7.702900,22.183800,22.175000,18.816500


In [ ]:
trainer.train()